In [1]:
import warnings
warnings.simplefilter("ignore")

import json
import pandas as pd
from tensorflow.keras.models import load_model

In [2]:
with open('test_review.json', encoding='utf-8') as f:
    test_reviews = json.loads(f.readline())

In [3]:
test_reviews

[{'business_id': '5ba3c04d-0841-45e3-89ca-9be71336c229',
  'date': '2021-08-20',
  'review_id': 'd6667ffd-36ef-4244-8776-a5032407dc1d',
  'stars': 5,
  'user_id': '4888afc0-06a2-425a-9d9a-630148912211'},
 {'business_id': '4451391d-88df-4fba-a411-a0fe7fa627e6',
  'date': '2021-06-05',
  'review_id': '29f4c7d5-10f0-4145-93d1-154dd95ed133',
  'stars': 5,
  'user_id': 'f07f20c9-5dc2-4bd7-bba5-ef2491bd9911'},
 {'business_id': '3b8df1df-92b8-4f37-bce0-c0bc755f99ca',
  'date': '2021-02-24',
  'review_id': '0607965b-bb0c-423c-8ac8-e12bcb19f9d4',
  'stars': 4,
  'user_id': '4992f483-0cf8-4c01-8f14-ec5a68fc70af'},
 {'business_id': '5e2f05d1-7238-4030-9645-bbdcab5cecf4',
  'date': '2021-12-18',
  'review_id': 'f51d3f61-8077-4f4c-bf57-0730d5e6c5ff',
  'stars': 1,
  'user_id': '271fd288-22db-4059-90f7-f5de85d62db9'},
 {'business_id': '3d4ca5bb-7416-40ec-a176-41d43dc601f4',
  'date': '2021-09-09',
  'review_id': '17a88ab4-9ecb-41f6-a27a-c4602bf3459d',
  'stars': 1,
  'user_id': '4992f483-0cf8-4c01-8

In [4]:
with open('business_clusters.json') as f:
    test = json.load(f)
test

[{'business_id': '5ba3c04d-0841-45e3-89ca-9be71336c229', 'label': 1},
 {'business_id': '4451391d-88df-4fba-a411-a0fe7fa627e6', 'label': 1},
 {'business_id': '3b8df1df-92b8-4f37-bce0-c0bc755f99ca', 'label': 1},
 {'business_id': '5e2f05d1-7238-4030-9645-bbdcab5cecf4', 'label': 0},
 {'business_id': '3d4ca5bb-7416-40ec-a176-41d43dc601f4', 'label': 0},
 {'business_id': '8af68c91-b1d3-43f9-a6ca-18451cd857f7', 'label': 0},
 {'business_id': '4bbb707c-dbde-4241-a249-d2748bcd3ec8', 'label': 0},
 {'business_id': '073f564c-22de-4cd4-9c44-9da9266fc43f', 'label': 1},
 {'business_id': 'a7d65ec8-ab5e-4a98-98f2-e6a873c4b882', 'label': 1},
 {'business_id': '58bba2e4-e48b-4440-a500-79b03e933ab0', 'label': 1},
 {'business_id': '3f07d8cc-46fd-4950-a86e-1e484be3564e', 'label': 1},
 {'business_id': 'e1b876f7-28ea-48d7-a965-9c6b05c93929', 'label': 0},
 {'business_id': '4625bc3a-f655-4893-a82a-32daa061eee4', 'label': 1},
 {'business_id': 'a143eff8-7676-4b41-a178-46a7e6bdc423', 'label': 1},
 {'business_id': '1f

In [5]:
df = pd.DataFrame(test_reviews) \
    .merge(pd.DataFrame(test), how='left') \
    .fillna({'label': -1}) \
    .groupby(['user_id','label']).mean()['stars'].reset_index() \
    .pivot(columns='label', values='stars', index='user_id') \
    .drop(-1, axis='columns') \
    .fillna(3)

for i in range(36):
    if i not in df.columns:
        df[i] = 3
    

In [6]:
df

label,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0
user_id,,,,,,,,,,,,,,,,,,,,,
271fd288-22db-4059-90f7-f5de85d62db9,2.285714,2.4,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
34b73ec2-d887-438e-89a9-a82affb68876,3.000000,3.5,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4888afc0-06a2-425a-9d9a-630148912211,3.000000,3.6,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4992f483-0cf8-4c01-8f14-ec5a68fc70af,2.571429,3.5,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
f07f20c9-5dc2-4bd7-bba5-ef2491bd9911,4.000000,2.9,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


In [7]:
model = load_model('yelp_model.h5')

In [8]:
model.predict(df)

array([[2.7819903],
       [3.519122 ],
       [3.5769267],
       [3.4583547],
       [3.3140836]], dtype=float32)